# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [6]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2332489293886,
 'week52high': 190,
 'week52low': 129.71,
 'week52highSplitAdjustOnly': 183.02,
 'week52lowSplitAdjustOnly': 135.37,
 'week52change': -0.014298392480484997,
 'sharesOutstanding': 16146209211,
 'float': 0,
 'avg10Volume': 97732440,
 'avg30Volume': 104283447,
 'day200MovingAvg': 158.51,
 'day50MovingAvg': 157.56,
 'employees': 151908,
 'ttmEPS': 6.1,
 'ttmDividendRate': 0.905442743894323,
 'dividendYield': 0.006475322468759705,
 'nextDividendDate': '',
 'exDividendDate': '2022-07-29',
 'nextEarningsDate': '2022-10-23',
 'peRatio': 22.78647846979869,
 'beta': 1.294168967631439,
 'maxChangePercent': 55.348190563938964,
 'year5ChangePercent': 2.8525172958164813,
 'year2ChangePercent': 0.24001653736273146,
 'year1ChangePercent': -0.01451564597498533,
 'ytdChangePercent': -0.21715056782785452,
 'month6ChangePercent': -0.1779208737241615,
 'month3ChangePercent': -0.04808873796377755,
 'month1ChangePercent': -0.09358623894341282,
 'day30

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [12]:
data['year1ChangePercent']

-0.01451564597498533

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [14]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [23]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        if(symbol in data):
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [
                        symbol, data[symbol]['price'],
                        data[symbol]['stats']['year1ChangePercent'], 'N/A'
                    ], index = my_columns
                ), ignore_index = True
            )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,131.40,-0.163721,N/A
1,AAL,12.72,-0.412429,N/A
2,AAP,164.92,-0.234902,N/A
3,AAPL,141.18,-0.014547,N/A
4,ABBV,141.69,0.299848,N/A
...,...,...,...,...
496,YUM,111.66,-0.101948,N/A
497,ZBH,110.79,-0.258311,N/A
498,ZBRA,274.84,-0.48117,N/A
499,ZION,51.06,-0.191858,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [30]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,83.25,2.296969,N/A
1,355,OXY,72.70,1.103582,N/A
2,147,DVN,72.88,0.935661,N/A
3,42,APA,43.46,0.802993,N/A
4,89,CF,110.00,0.777122,N/A
5,298,MCK,358.01,0.776195,N/A
6,315,MRO,28.28,0.765746,N/A
7,492,XOM,105.99,0.716361,N/A
8,313,MPC,110.81,0.689515,N/A
9,109,COP,119.86,0.671593,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [34]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    try:
        portfolio_size= float(portfolio_size)
    except ValueError:
        print('Please enter a number value')
        portfolio_size = input('Enter the size of your portfolio: ')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 10000000
10000000.0


In [39]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])


/home/shahriar/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [157]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        if(symbol in data):
            hqm_dataframe = hqm_dataframe.append(
                pd.Series(
                    [
                        symbol,
                        data[symbol]['price'],
                        'N/A',
                        data[symbol]['stats']['year1ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month6ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month3ChangePercent'],
                        'N/A',
                        data[symbol]['stats']['month1ChangePercent'],
                        'N/A',
                        'N/A',
                    ], index=hqm_columns
                ), ignore_index=True
            )
  

## Cleaning Data
Clean all rows that have no historical Price Return, as in Nonetype

In [158]:
for row in hqm_dataframe.index:
    if(hqm_dataframe.loc[row, 'One-Year Price Return'] == None):
        hqm_dataframe.drop(row, inplace=True)

hqm_dataframe.reset_index(inplace=True, drop=True)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [159]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month',   
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.84,N/A,-0.1661,45.674044,-0.052423,75.050302,0.043116,78.269618,-0.057246,67.605634,N/A
1,AAL,12.20,N/A,-0.402891,11.066398,-0.269341,21.529175,-0.130069,24.144869,-0.131317,28.169014,N/A
2,AAP,161.74,N/A,-0.226674,34.808853,-0.275164,20.321932,-0.11438,28.973843,-0.101858,42.857143,N/A
3,AAPL,143.68,N/A,-0.014802,70.82495,-0.179983,46.076459,-0.046929,51.509054,-0.094849,47.484909,N/A
4,ABBV,144.76,N/A,0.301253,93.158954,-0.197179,41.851107,-0.084705,37.22334,-0.013119,84.507042,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,111.67,N/A,-0.104389,57.545272,-0.079123,69.416499,-0.069872,42.655936,-0.06563,62.977867,N/A
493,ZBH,111.73,N/A,-0.263378,28.973843,-0.165823,51.911469,0.03017,75.251509,-0.02068,83.098592,N/A
494,ZBRA,271.54,N/A,-0.469729,5.83501,-0.359392,8.651911,-0.113429,29.17505,-0.107697,39.43662,N/A
495,ZION,51.54,N/A,-0.191952,42.052314,-0.206425,39.839034,-0.006395,63.179074,-0.14164,23.943662,N/A


In [160]:
hqm_dataframe.loc[0, 'One-Year Return Percentile']

45.67404426559356

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [161]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)   

In [162]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.84,N/A,-0.1661,45.674044,-0.052423,75.050302,0.043116,78.269618,-0.057246,67.605634,66.649899
1,AAL,12.20,N/A,-0.402891,11.066398,-0.269341,21.529175,-0.130069,24.144869,-0.131317,28.169014,21.227364
2,AAP,161.74,N/A,-0.226674,34.808853,-0.275164,20.321932,-0.11438,28.973843,-0.101858,42.857143,31.740443
3,AAPL,143.68,N/A,-0.014802,70.82495,-0.179983,46.076459,-0.046929,51.509054,-0.094849,47.484909,53.973843
4,ABBV,144.76,N/A,0.301253,93.158954,-0.197179,41.851107,-0.084705,37.22334,-0.013119,84.507042,64.185111
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,111.67,N/A,-0.104389,57.545272,-0.079123,69.416499,-0.069872,42.655936,-0.06563,62.977867,58.148893
493,ZBH,111.73,N/A,-0.263378,28.973843,-0.165823,51.911469,0.03017,75.251509,-0.02068,83.098592,59.808853
494,ZBRA,271.54,N/A,-0.469729,5.83501,-0.359392,8.651911,-0.113429,29.17505,-0.107697,39.43662,20.774648
495,ZION,51.54,N/A,-0.191952,42.052314,-0.206425,39.839034,-0.006395,63.179074,-0.14164,23.943662,42.253521


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [163]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,COP,120.30,N/A,0.677045,98.189135,0.191187,98.189135,0.397291,99.798793,0.112016,98.390342,98.641851
1,MPC,111.53,N/A,0.687114,98.390342,0.24779,98.792757,0.300141,98.189135,0.089476,96.981891,98.088531
2,LB,81.83,N/A,2.307106,100.0,0.850757,100.0,0.218839,95.171026,0.081569,96.177062,97.837022
3,DVN,75.53,N/A,0.941695,99.597586,0.18148,97.384306,0.328712,98.993964,0.050042,94.969819,97.736419
4,UNM,43.18,N/A,0.552138,97.183099,0.324334,99.597586,0.257513,97.183099,0.086426,96.579477,97.635815
5,MRO,29.22,N/A,0.794389,99.396378,0.088012,94.366197,0.304321,98.591549,0.091517,97.183099,97.384306
6,COG,22.65,N/A,0.344307,93.963783,0.181088,97.183099,0.267543,97.585513,0.259356,100.0,97.183099
7,HES,129.90,N/A,0.468346,95.975855,0.144163,96.780684,0.292026,97.987928,0.089108,96.780684,96.881288
8,XOM,106.07,N/A,0.720137,98.591549,0.18904,97.78672,0.193986,94.768612,0.067352,95.372233,96.629779
9,OXY,73.00,N/A,1.138819,99.798793,0.137244,96.579477,0.156321,91.95171,0.082746,96.37827,96.177062


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [164]:
portfolio_input()

Enter the size of your portfolio: 10000000


In [165]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

/home/shahriar/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [166]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,COP,120.30,1662,0.677045,98.189135,0.191187,98.189135,0.397291,99.798793,0.112016,98.390342,98.641851
1,MPC,111.53,1793,0.687114,98.390342,0.24779,98.792757,0.300141,98.189135,0.089476,96.981891,98.088531
2,LB,81.83,2444,2.307106,100.0,0.850757,100.0,0.218839,95.171026,0.081569,96.177062,97.837022
3,DVN,75.53,2647,0.941695,99.597586,0.18148,97.384306,0.328712,98.993964,0.050042,94.969819,97.736419
4,UNM,43.18,4631,0.552138,97.183099,0.324334,99.597586,0.257513,97.183099,0.086426,96.579477,97.635815
5,MRO,29.22,6844,0.794389,99.396378,0.088012,94.366197,0.304321,98.591549,0.091517,97.183099,97.384306
6,COG,22.65,8830,0.344307,93.963783,0.181088,97.183099,0.267543,97.585513,0.259356,100.0,97.183099
7,HES,129.90,1539,0.468346,95.975855,0.144163,96.780684,0.292026,97.987928,0.089108,96.780684,96.881288
8,XOM,106.07,1885,0.720137,98.591549,0.18904,97.78672,0.193986,94.768612,0.067352,95.372233,96.629779
9,OXY,73.00,2739,1.138819,99.798793,0.137244,96.579477,0.156321,91.95171,0.082746,96.37827,96.177062


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

## Saving Our Excel Output

As before, saving our Excel output is very easy: